In [1]:
# !! RUN THIS CELL FIRST !! It sets up a global function for use in the rest of the notebook

# Here is the convenience function that throws an exception for us 
function throwFunc { throw "This is an exception!" }

# This sets it to be available across all the cells in this notebook
#!set --name throwFunc --value throwFunc

## Exceptions
Exceptions are errors in code that generally halt execution; these are known as terminating exceptions. Such errors can be 'caught' using `try`/`catch` blocks (explained in the next section). 

The utility function `throwFunc` at the top of this notebook contains a `throw` block which generates an exception manually.

Errors generated by a `throw` block are terminating, however errors generated by `Write-Error` cmdlet are not; errors generated by Write-Error are just written to the error output stream (although a terminating error can be forced with the `-ErrorAction stop` parameter).

In [2]:
"+++ throwFunc terminates execution if the exception it throws is not caught +++"
throwFunc
"+++ Because a throw block is terminating, this should not print +++"

+++ throwFunc terminates execution if the exception it throws is not caught +++
Exception: 
Line |
   4 |  function throwFunc { throw "This is an exception!" }
     |                       ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | This is an exception!


Error: Command failed: SubmitCode: "+++ throwFunc terminates execution if the excepti ...

In [3]:
Write-Error "An error is written to the error stream..."
"...but it doesnt stop execution."
Write-Error "This does stop execution though" -ErrorAction Stop
"+++ This wont print +++"

Write-Error: An error is written to the error stream...
...but it doesnt stop execution.
Write-Error: 
Line |
   3 |  Write-Error "This does stop execution though" -ErrorAction Stop
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | This does stop execution though


Error: Command failed: SubmitCode: Write-Error "An error is written to the error stre ...

## Try/Catch

PowerShell handles exceptions using `try`/`catch` blocks.

A `try` block runs until a terminating exception is generated, at which point control is passed to the `catch` block in order to process the exception.

In [4]:
try {
    throwFunc
} catch {
    Write-Output "I only get printed if there is a terminating error in the try block"
    Write-Error $_
}


I only get printed if there is a terminating error in the try block
Exception: This is an exception!


Error: Command failed: SubmitCode: try { ...

## The Call Stack

Functions and cmdlets can call other functions and cmdlets within PowerShell code, but when an exception happens not every function involved will be provided in the exception message. This can make it difficult to  understand the full chain of functions that led to an exception. For example, the below wrapper function `A` contains the throwFunc function which generates a terminating exception, however the functions that led to `A` are not included in the exception message:

In [5]:
function A {
    throwFunc
}
function B {
    A
}
function C {
    B
}
C

Exception: 
Line |
   4 |  function throwFunc { throw "This is an exception!" }
     |                       ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | This is an exception!


Error: Command failed: SubmitCode: function A { ...

The _call stack_ is a stack structure (so first in firs out) of functions that are currently executing. The `$PSItem` automatic variable contains the full call stack, which can be examined by printing the `ScriptStackTrace` property.

In [7]:
function A {
    throwFunc
}
function B {
    A
}
function C {
    B
}

try {
    C
} catch {
    $PSItem.ScriptStackTrace
}

at throwFunc, <No file>: line 4
at A, <No file>: line 2
at B, <No file>: line 5
at C, <No file>: line 8
at <ScriptBlock>, <No file>: line 12


Error: Command failed: SubmitCode: function A { ...